In [1]:
import sys

sys.path.append(".")
import joblib
import pickle
import argparse
from lang import *
from novelty.han.han_novelty import *
from snli.bilstm.bilstm import *
from snli.attn_enc.attn_enc import *
from novelty.train_utils import *
import shutil
from utils import (
    load_bilstm_encoder,
    load_attn_encoder,
    load_han_clf_encoder,
    load_han_reg_encoder,
    reset_model,
    seed_torch,
)
import numpy as np
from novelty.train_utils import *
from datamodule import *
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from utils.keys import NEPTUNE_API
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tqdm import tqdm
import random
import copy
import neptune
from pytorch_lightning.metrics import Accuracy,F1,Precision,Recall



In [2]:
def train(model, data_module, optimizer, device):
    model.train()
    loss_values = []
    for batch in tqdm(data_module.train_dataloader()):
        x0, x1, y = batch
        model.zero_grad()
        opt = model(x0.to(device), x1.to(device)).squeeze(1)
        loss = F.cross_entropy(opt, y.to(device))
        loss.backward()
        loss_values.append(loss.cpu().item())
        optimizer.step()
    return np.mean(loss_values)

def evaluate(model, data_module, device):
    loss_values = []
    accuracy_values = []
    precision_values = []
    recall_values = []
    f1_values = []
    acc_metric = Accuracy(num_classes=2)
    prec_metric = Precision(num_classes=2)
    recall_metric = Recall(num_classes=2)
    F1_metric = F1(num_classes=2)

    model.eval()
    with torch.no_grad():
        for batch in tqdm(data_module.test_dataloader()):
            x0, x1, y = batch
            model.zero_grad()
            opt = model(x0.to(device), x1.to(device)).squeeze(1)
            loss = F.cross_entropy(opt, y.to(device))
            pred = F.softmax(opt)
            loss_values.append(loss.cpu().item())
            accuracy_values.append(acc_metric(pred.argmax(1).cpu(),y.cpu()))
            precision_values.append(prec_metric(pred.argmax(1).cpu(),y.cpu()))
            recall_values.append(recall_metric(pred.argmax(1).cpu(),y.cpu()))
            f1_values.append(F1_metric(pred.argmax(1).cpu(),y.cpu()))

    print(accuracy_values)
    return (
        np.mean(loss_values),
        np.mean(accuracy_values),
        np.mean(precision_values),
        np.mean(recall_values),
        np.mean(f1_values),
    )

In [3]:
seed_torch(140)

In [4]:
encoder_ = 'clf'
if encoder_ == "reg":
    encoder, Lang = load_han_reg_encoder()
elif encoder_ == "clf":
    encoder, Lang = load_han_clf_encoder()


In [5]:
webis = False
data_module = (
    WebisDataModule(batch_size=32, cross_val=True)
    if webis
    else DLNDDataModule(batch_size=32, cross_val=True)
)

In [6]:
data_module.prepare_data(Lang, 100)

In [7]:
hparams = {
    "optim": "adamw",
    "weight_decay": 0.1,
    "lr": 0.00010869262115700171,
    "scheduler": "lambda",
}

In [8]:
model_conf = HAN_Novelty_conf(encoder, **hparams)
model = HAN_Novelty(model_conf)


In [9]:
optimizer = optim.AdamW(model.parameters(), lr=hparams["lr"],weight_decay = hparams["weight_decay"])
init_state = copy.deepcopy(model.state_dict())
init_state_opt = copy.deepcopy(optimizer.state_dict())

In [10]:
data_module.set_fold(0)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

HAN_Novelty(
  (doc_encoder): HAN(
    (encoder): Attn_Encoder(
      (embedding): Embedding(30522, 300, padding_idx=0)
      (translate): Linear(in_features=300, out_features=400, bias=True)
      (act): Tanh()
      (lstm_layer): LSTM(400, 400, num_layers=2, bidirectional=True)
      (attention): Attention(
        (Ws): Linear(in_features=800, out_features=100, bias=False)
        (Wa): Linear(in_features=100, out_features=1, bias=False)
      )
    )
    (translate): Linear(in_features=800, out_features=300, bias=True)
    (act): ReLU()
    (lstm_layer): LSTM(300, 300, num_layers=2, bidirectional=True)
    (attention): Attention(
      (Ws): Linear(in_features=600, out_features=50, bias=False)
      (Wa): Linear(in_features=50, out_features=1, bias=False)
    )
  )
  (fc_in): Linear(in_features=2400, out_features=600, bias=True)
  (act): ReLU()
  (fc_out): Linear(in_features=600, out_features=2, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
)

In [20]:
EPOCHS = 5
for ep in range(EPOCHS):
    train_loss = train(model, data_module, optimizer, device)
    print(f"\tTraining Loss => epoch {ep}: {train_loss}")
    test_loss, test_acc, test_prec, test_recall, test_f1 = evaluate(model, data_module, device)
    print(f"\t\t Loss : {test_loss}, Accuracy: {test_acc}, Precsion: {test_prec}, Recall: {test_recall}, F1 Score:{test_f1}")

100%|██████████| 17/17 [00:12<00:00,  1.37it/s]		 Loss : 2.1262640953063965, Accuracy: 0.4534313678741455, Precsion: 0.4466345129257464, Recall: 0.4489255863675368, F1 Score:0.43898291283087093



In [27]:
test_loss, test_acc, test_prec, test_recall, test_f1 = evaluate(model, data_module, device)
print(f"\t\t Loss : {test_loss}, Accuracy: {test_acc}, Precsion: {test_prec}, Recall: {test_recall}, F1 Score:{test_f1}")

100%|██████████| 17/17 [00:12<00:00,  1.36it/s][tensor(0.4375), tensor(0.4062), tensor(0.4375), tensor(0.7812), tensor(0.5000), tensor(0.2500), tensor(0.4062), tensor(0.5000), tensor(0.3438), tensor(0.4688), tensor(0.5625), tensor(0.3125), tensor(0.5625), tensor(0.5000), tensor(0.4062), tensor(0.4375), tensor(0.3750)]
		 Loss : 2.115576751091901, Accuracy: 0.4522058963775635, Precsion: 0.4524940848350525, Recall: 0.4556811451911926, F1 Score:0.43799063563346863



In [26]:
from pytorch_lightning.metrics import Accuracy,F1,Precision,Recall

def evaluate(model, data_module, device):
    loss_values = []
    accuracy_values = []
    precision_values = []
    recall_values = []
    f1_values = []
    acc_metric = Accuracy(num_classes=2)
    prec_metric = Precision(num_classes=2)
    recall_metric = Recall(num_classes=2)
    F1_metric = F1(num_classes=2)

    model.eval()
    with torch.no_grad():
        for batch in tqdm(data_module.test_dataloader()):
            x0, x1, y = batch
            model.zero_grad()
            opt = model(x0.to(device), x1.to(device)).squeeze(1)
            loss = F.cross_entropy(opt, y.to(device))
            pred = F.softmax(opt)

            loss_values.append(loss.cpu().item())

            accuracy_values.append(
                acc_metric(pred.argmax(1).cpu(),y.cpu())
            )
            precision_values.append(prec_metric(pred.argmax(1).cpu(),y.cpu()))
            recall_values.append(recall_metric(pred.argmax(1).cpu(),y.cpu()))
            f1_values.append(F1_metric(pred.argmax(1).cpu(),y.cpu()))

    print(accuracy_values)
    return (
        np.mean(loss_values),
        np.mean(accuracy_values),
        np.mean(precision_values),
        np.mean(recall_values),
        np.mean(f1_values),
    )

In [28]:
with torch.no_grad():
    for batch in tqdm(data_module.test_dataloader()):
        x0,x1,y = batch
        opt = model(x0.to(device), x1.to(device)).squeeze(1)
        loss = F.cross_entropy(opt, y.to(device))
        pred = F.softmax(opt)
        print(opt,pred,y)
        break


  0%|          | 0/17 [00:01<?, ?it/s]tensor([[-1.5441,  1.5205],
        [ 4.6311, -4.4921],
        [-0.7040,  0.6580],
        [ 3.7006, -3.5853],
        [ 0.1015, -0.1009],
        [ 3.7428, -3.6935],
        [ 4.3301, -4.2176],
        [-1.3374,  1.3062],
        [-0.5537,  0.5221],
        [ 0.5367, -0.5602],
        [-1.5412,  1.4964],
        [ 0.0832, -0.1174],
        [ 1.7061, -1.6620],
        [-0.3722,  0.3431],
        [ 4.5172, -4.3984],
        [-0.2374,  0.2032],
        [ 0.2591, -0.2440],
        [ 1.7056, -1.6910],
        [-0.2993,  0.2478],
        [ 2.3985, -2.3388],
        [ 0.1920, -0.2103],
        [-0.8371,  0.8153],
        [-1.3374,  1.3062],
        [ 3.6410, -3.5456],
        [-1.3374,  1.3062],
        [-1.7551,  1.6844],
        [ 1.0876, -1.0989],
        [-0.9657,  0.9343],
        [-0.5913,  0.5412],
        [ 3.2502, -3.1724],
        [ 1.2195, -1.2033],
        [-1.2263,  1.1789]], device='cuda:0') tensor([[4.4589e-02, 9.5541e-01],
        [9.998

In [8]:
import svn.remote

r = svn.remote.RemoteClient('http://svn.dridan.com/sandpit/QA/trecdata/datacollection/')
r.checkout('/content/DeepDocumentNovelty/trecdatasvn')